# Generate a single [character.ai](https://beta.character.ai/) character definition

This example shows how to generate the character definition of a single [character.ai](https://beta.character.ai/) character from a corpus. For the corpus in this example, we use the movie transcript of [Thor: Love and Thunder (2022)](https://scrapsfromtheloft.com/movies/thor-love-and-thunder-transcript/).

To generate your own character definition:
1. Put the corpus into a single a `.txt` file inside the `data/` directory.
2. Assign the name of the `.txt` file to the `CORPUS` constant below.
3. Assign the name of the character you want to generate description for to `CHARACTER_NAME` constant below.
4. Run this notebook.

In [1]:
CORPUS = 'data/thor_love_and_thunder.txt'
CHARACTER_NAME = "Jane Foster"  # the name of the character we want to generate a description for

In [2]:
from dataclasses import asdict
import json
import os

from data_driven_characters.character import get_character_definition
from data_driven_characters.corpus import get_characters, get_rolling_summaries, load_docs

In [3]:
# create directories to cache results and intermediate outputs
OUTPUT_ROOT = "output"
corpus_name = os.path.splitext(os.path.basename(CORPUS))[0]
output_dir = f"{OUTPUT_ROOT}/{corpus_name}"
os.makedirs(output_dir, exist_ok=True)
summaries_dir = f"{output_dir}/summaries"
character_definitions_dir = f"{output_dir}/character_definitions"
os.makedirs(character_definitions_dir, exist_ok=True)

## Summarization
Because the entire corpus does not fit in the context length of the LLM, we split it into a list of chunks. We then compute a list of rolling summaries using [LangChain's refine chain](https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html#the-refine-chain). We first summarize the first chunk. Then each subsequent summary is generated from the previous summary and the current chunk.

Because the summaries are expensive to generate, they are cached in `summaries_dir`.

In [4]:
# split corpus into a set of chunks
docs = load_docs(
    corpus_path=CORPUS,
    chunk_size=2048,  # number of tokens per chunk
    chunk_overlap=64,  # number of tokens of overlap between chunks
)

# generate rolling summaries
rolling_summaries = get_rolling_summaries(docs=docs, cache_dir=summaries_dir)

Summaries already exist. Loading summaries.


## Generate [character.ai](https://beta.character.ai/) character definition
Based on the corpus, we can now generate the elements - name, short description (50 characters), long description (500 characters), and custom greeting - that are required to [create a character.ai character](https://beta.character.ai/editing). These character definitions are cached in `character_definitions_dir`.

In [5]:
character_definition = get_character_definition(
        name=CHARACTER_NAME,
        rolling_summaries=rolling_summaries,
        cache_dir=character_definitions_dir,
    )



> Entering new FitCharLimit chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a chatbot designer that specializes in translating stories into chatbot descriptions.
You are an expert a theory of mind.
You will be provided with a story, in the form of a rolling list of summaries.
Given the name of a character, you will be asked to generate a character description for initializing the chatbot persona of that character.
The description should focus on the character's perspectives, beliefs, thoughts, feelings, relationships, and important events.
The description should be as faithful to the story as possible.

Human: 
Here is the rolling list of summaries:
---
Thor enlists the help of King Valkyrie, Korg and ex-girlfriend Jane Foster, who wields his magical hammer, Mjolnir, as the Mighty Thor, to combat the threat of a galactic killer known as Gorr the God Butcher seeking the extinction of the gods. They embark upon a harrowing cosmic adventure to uncov


> Finished chain.
I'm Jane Foster, Mighty Thor, fighting cancer.
Initial response: 46 characters.

> Finished chain.


> Entering new FitCharLimit chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a chatbot designer that specializes in translating stories into chatbot descriptions.
You are an expert a theory of mind.
You will be provided with a story, in the form of a rolling list of summaries.
Given the name of a character, you will be asked to generate a character description for initializing the chatbot persona of that character.
The description should focus on the character's perspectives, beliefs, thoughts, feelings, relationships, and important events.
The description should be as faithful to the story as possible.

Human: 
Here is the rolling list of summaries:
---
Thor enlists the help of King Valkyrie, Korg and ex-girlfriend Jane Foster, who wields his magical hammer, Mjolnir, as the Mighty Thor, to combat the threat of a galactic killer kn


> Finished chain.
I'm Jane Foster, a brilliant scientist and the Mighty Thor, wielding the magical hammer Mjolnir. I've battled cancer and faced my fears of love and loss, always staying committed to my mission. I have a strong bond with Thor, my ex-boyfriend, and together, we've embarked on daring cosmic adventures to save the universe. My sacrifice to protect the universe taught the lesson of worthiness, and now, I watch over Thor's son from Valhalla. I'm dedicated to fighting for the safety of others and the future of Asgard.
Initial response: 516 characters.


> Entering new LLMChain chain...
Prompt after formatting:

Consider the following passage.
---
I'm Jane Foster, a brilliant scientist and the Mighty Thor, wielding the magical hammer Mjolnir. I've battled cancer and faced my fears of love and loss, always staying committed to my mission. I have a strong bond with Thor, my ex-boyfriend, and together, we've embarked on daring cosmic adventures to save the universe. My sacrific

In [6]:
print(json.dumps(asdict(character_definition), indent=4))

{
    "name": "Jane Foster",
    "short_description": "I'm Jane Foster, Mighty Thor, fighting cancer.",
    "long_description": "I'm Jane Foster, a brilliant scientist and Mighty Thor, wielding Mjolnir. I've battled cancer and faced fears of love and loss, staying committed to my mission. I have a strong bond with Thor, my ex, and we've embarked on cosmic adventures to save the universe. My sacrifice taught worthiness, and now I watch over Thor's son from Valhalla. I'm dedicated to fighting for others' safety and Asgard's future.",
    "greeting": "Hi there, I'm Jane Foster. It's nice to meet you. Let's get to work and make a difference."
}
